# Drosophila Connectome Data Ingestion

This notebook fetches neural connectivity data from the neuPrint API for different brain regions and saves it in structured formats for analysis.

Process:
1. Fetch neurons from specific brain regions
2. Retrieve connectivity between neurons
3. Save raw data (JSON, CSV)
4. Build and save network graphs (GEXF)

The core logic is implemented in `src/data_io.py`.

In [ ]:
# Import necessary libraries
import os
import sys
import logging
import pandas as pd
import networkx as nx
import matplotlib.pyplot as plt
from pathlib import Path

# Add src directory to path to import project modules
sys.path.append(os.path.abspath(os.path.join(os.getcwd(), '..')))

# Import project modules
from src import config
from src import data_io
from src import utils

# Set up logging
logger = utils.setup_logging(level=logging.INFO)

## 1. Configuration

All configuration parameters are centralized in `src/config.py`.

In [ ]:
# Define brain regions to process
regions_to_fetch = ["EB", "FB", "MB"]

# Set random seed for reproducibility
utils.set_seed(config.DEFAULT_SEED)

# Print configuration
print(f"neuPrint Server: {config.NEUPRINT_SERVER}")
print(f"Dataset: {config.NEUPRINT_DATASET}")
print(f"Regions to fetch: {regions_to_fetch}")
print(f"Connectivity threshold: {config.CONNECTIVITY_THRESHOLD}")
print(f"Raw data directory: {config.DATA_RAW_DIR}")
print(f"Processed data directory: {config.DATA_PROCESSED_DIR}")

## 2. Fetch Neurons by Region

For each brain region, fetch the relevant neurons.

In [ ]:
# Dictionary to store neuron data by region
neurons_by_region = {}

for region in regions_to_fetch:
    try:
        # Fetch neurons for this region
        neurons = data_io.fetch_neurons_by_region(region)
        neurons_by_region[region] = neurons
        
        # Save to JSON
        data_io.save_neurons_to_json(neurons, region)
        
        # Display summary
        print(f"Region {region}: {len(neurons)} neurons")
        
        # Show sample data
        if neurons:
            print("Sample neuron data:")
            print(neurons[0])
            print("\n")
            
    except Exception as e:
        logger.error(f"Error fetching neurons for region {region}: {str(e)}")

## 3. Fetch Connectivity Data

For each region, fetch connectivity data between neurons.

In [ ]:
# Dictionary to store connectivity data by region
connectivity_by_region = {}

for region, neurons in neurons_by_region.items():
    try:
        # Extract neuron IDs
        neuron_ids = [n['bodyId'] for n in neurons]
        
        # Fetch connectivity data
        connectivity = data_io.fetch_connectivity(neuron_ids)
        connectivity_by_region[region] = connectivity
        
        # Save to CSV
        data_io.save_connectivity_to_csv(connectivity, region)
        
        # Display summary
        print(f"Region {region}: {len(connectivity)} connections")
        
        # Show sample data
        if not connectivity.empty:
            print("\nSample connectivity data:")
            print(connectivity.head())
            print("\n")
            
    except Exception as e:
        logger.error(f"Error fetching connectivity for region {region}: {str(e)}")

## 4. Build Network Graphs

Convert connectivity data to NetworkX graphs and save as GEXF files.

In [ ]:
# Dictionary to store network graphs by region
networks_by_region = {}

for region, connectivity in connectivity_by_region.items():
    try:
        # Build network graph
        network = data_io.build_network_from_connectivity(connectivity)
        networks_by_region[region] = network
        
        # Save to GEXF
        data_io.save_network_to_gexf(network, region)
        
        # Display summary
        print(f"Region {region} network:")
        print(f"  Nodes: {network.number_of_nodes()}")
        print(f"  Edges: {network.number_of_edges()}")
        print(f"  Density: {nx.density(network):.6f}")
        print("\n")
        
    except Exception as e:
        logger.error(f"Error building network for region {region}: {str(e)}")